In [1]:
import numpy as np
import pandas as pd
import os

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from string import punctuation
from collections import Counter

import tensorflow as tf
from tensorflow.python.keras import Model
from tensorflow.python.keras.layers import Embedding, LSTM, Dense, Dropout

from utils.DataLoader import DataLoader



In [2]:
def rearrange_data(train_test, label):
    content = []
    for filepath in os.listdir('data/' + train_test + '/' + label):
        with open(('data/' + train_test + '/' + label + '/' + filepath), 'r') as file:
            review = file.read()
            content.append(review)
    with open('data/' + train_test + '/' + label + '.txt', 'w') as target_file:
        target_file.write('\n'.join(content))
        print(target_file.name)


rearrange_data('train', 'pos')
rearrange_data('train', 'neg')

rearrange_data('test', 'pos')
rearrange_data('test', 'neg')





data/train/pos.txt


data/train/neg.txt


data/test/pos.txt


data/test/neg.txt


In [3]:
dataloader = DataLoader(8)


In [9]:
class SentimentLSTM(Model):
    def __init__(self, vocab_size, output_size, embedding_dim, lstm_layers):
        super(SentimentLSTM, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim, input_length=200)
        self.lstm1= LSTM(lstm_layers, dropout=0.2, activation='tanh', return_sequences=True)
        self.lstm2 = LSTM(lstm_layers, dropout=0.2, activation='tanh')
        self.dropout = Dropout(0.3)
        self.d2 = Dense(output_size, activation='sigmoid')

    def call(self, x):
        x = self.embedding(x)
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.d1(x)
        return self.d2(x)


vocab_size = len(dataloader.vocab_to_int) + 1 # +1 for the 0 padding
output_size = 1
embedding_dim = 400
lstm_layers = 256
net = SentimentLSTM(vocab_size, output_size, embedding_dim, lstm_layers)
print(net)